## Initialize

In [ ]:
#library(Rmisc)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

dataset_name = "210714_metabolomics"
path = "/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb"
data_path = glue("{base_path}/data")
dataset_path = glue("{data_path}/3_datasets_post/{dataset_name}")

project_label="21_metabolomics_multitask"
project_path = glue("{base_path}/results/projects/{project_label}")
figures_path = glue("{project_path}/figures")
data_results_path = glue("{project_path}/data")

## Load data

In [ ]:
list.dirs(path = project_path, full.names = TRUE, recursive = TRUE)

In [ ]:
run = "220126_apoe4"

In [ ]:
DSM = "MultiTaskSurvivalTraining"
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) 
data_description = arrow::read_feather(glue("{dataset_path}/description_merged.feather"))
#predictions = arrow::read_feather(glue("{data_results_path}/predictions_210803_metabolomics.feather")) 
predictions = arrow::read_feather(glue("{data_results_path}/predictions_{run}_metabolomics_APOE4.feather")) 

In [ ]:
data_events = data %>% select(eid, ends_with("event"), ends_with("event_time")) %>% 
    pivot_longer(-eid, names_to=c("endpoint", "type"), values_to="value", names_pattern="(.*)(event_time|event)") %>% 
    mutate(endpoint = stringr::str_sub(endpoint, end=-2)) %>% pivot_wider(names_from="type", values_from="value")

In [ ]:
scores = c("DS_Metabolomics",
           "COX_PANEL",
           "COX_PANELapoe4", 
           "DS_PANEL+Metabolomics",
           "DS_PANELapoe4+Metabolomics")

In [ ]:
library(ggalt)

In [ ]:
library("jsonlite")
colors_path = "colors.json"
colors_dict = read_json(colors_path)

In [ ]:
color_map <- c("all" = "grey", "none" = "black",
               'DS_Metabolomics' = "#4F8EC1",  #colors_dict$pastel$orange$dark,
    
               'COX_PANEL' = colors_dict$pastel$grey$light, 
               'DS_PANEL+Metabolomics' = colors_dict$pastel$grey$mid, 
               
                'COX_PANELapoe4' = colors_dict$pastel$red$light,  
               'DS_PANELapoe4+Metabolomics' = colors_dict$pastel$red$mid  
      )

In [ ]:
scores_full = names(color_map)

In [ ]:
name = glue("benchmark_cindex_{run}")
benchmark_cindex_general = read_feather(glue("{data_results_path}/{name}.feather"))  %>% distinct() %>% unite("score", c(module, features), remove=FALSE) %>%  distinct()
   # mutate_at(vars(score), list(~ factor(., levels=scores_full)))

In [ ]:
base_size = 8
title_size = 8
facet_size = 8
geom_text_size=3
library(ggplot2); 
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=8), axis.text=element_text(size=8, color="black"), axis.text.x=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2)))

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

endpoint_order = c("M_MACE", "M_coronary_heart_disease", "M_cerebral_stroke", "M_all_cause_dementia", "M_heart_failure", "M_atrial_fibrillation",
                   "M_type_2_diabetes", "M_liver_disease", "M_renal_disease", "M_peripheral_arterial_disease", "M_venous_thrombosis",  "M_abdominal_aortic_aneurysm",
                   "M_chronic_obstructuve_pulmonary_disease", "M_asthma", 'M_parkinsons_disease', 'M_cataracts', 'M_glaucoma', 'M_fractures',
                    "M_lung_cancer","M_non_melanoma_skin_cancer","M_colon_cancer","M_rectal_cancer","M_prostate_cancer","M_breast_cancer"
                   
)

In [ ]:
em_wrap = endpoint_map
em_wrap[names(endpoint_map)] <- str_wrap(unname(endpoint_map), 20)
em_wrap

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
library(ggbeeswarm)
scores_plot = names(color_map)

temp = benchmark_cindex_general %>% 
    mutate_at(vars(score), list(~ factor(., levels=scores_full))) %>% filter(score==score) %>% filter(score %in% scores_plot)
temp_wide = temp %>% select(-module, -features) %>% pivot_wider(names_from="score", values_from="cindex") %>% mutate(delta=`DS_Metabolomics`-`COX_PANEL`)
temp_desc = temp %>% group_by(endpoint, score) %>% summarise(mean=median(cindex, na.rm=TRUE), max = max(cindex), .groups="drop")
temp_desc = temp_desc %>% select(-max) %>% pivot_wider(names_from="score", values_from="mean") %>% mutate(delta=`DS_Metabolomics`-`COX_PANEL`) %>% mutate(endpoint=fct_reorder(endpoint, desc(delta)))
endpoint_order_desc = levels(temp_desc$endpoint)

In [ ]:
library(ggdist)

In [ ]:
library(ggtext)

In [ ]:
library(ggforestplot)

In [ ]:
library(cowplot)

In [ ]:
scores_map = c(
    "DS_Metabolomics" = "MET",
    "COX_PANEL" = "PANEL", 
    "DS_PANEL+Metabolomics" = "PANEL+MET",
    "COX_PANELapoe4" = "PANEL(APOE4)", 
    "DS_PANELapoe4+Metabolomics" = "PANEL(APOE4)+MET"
               )

## Figure 3 - Performance in Context

In [ ]:
library(scales)

In [ ]:
scores_plot = c(
    "DS_Metabolomics",
    "COX_PANEL", 
    "DS_PANEL+Metabolomics",
    "COX_PANELapoe4",
    "DS_PANELapoe4+Metabolomics"
    
               )

endpoint_order = c("M_all_cause_dementia")


temp = benchmark_cindex_general %>% 
   mutate_at(vars(score), list(~ factor(., levels=scores_plot))) %>% filter(score==score) %>% filter(score %in% scores_plot) %>% 
    mutate(endpoint = factor(endpoint, levels=endpoint_order)) #%>% #filter(endpoint %in% endpoint_selection)
temp_desc = temp %>% group_by(endpoint, score) %>% summarise(mean=median(cindex, na.rm=TRUE), max = max(cindex), .groups="drop")

In [ ]:
## 66%, 95%
#endpoint_order_panel = (temp_wide %>% group_by(endpoint) %>% mutate(delta = `DS_PANEL+Metabolomics`-`COX_PANEL`) %>% summarise(delta=mean(delta)) %>% arrange(delta))$endpoint
plot_width=8; plot_height=2; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
met_panel_apoe4 = ggplot(temp_wide, aes(x=endpoint))+#levels=endpoint_order_desc))) +
    labs(x=NULL, y="\u0394 C-Index")+#, title="Discriminative Performance vs. PANEL(APOE4)")+#, title="Metabolomics (orange) vs. Age+Sex (grey) vs. AgeSexMetabolomic (red)")+
    geom_segment(aes(y=0, yend=0, x=0.5, xend=1.5), alpha=0.7, color="black", size=0.4, linetype=1)+

    stat_gradientinterval(mapping = aes(y=`COX_PANEL`-`COX_PANELapoe4`), 
                          alpha=1, fatten_point=0.8, interval_size_range=c(0.3, 0.6), 
                          position = position_nudge(x = +0.4), slab_alpha=0)+

    annotate("text", x=1.6, y=-0.0155, position = position_nudge(x = +1), label="PANEL", size=2)+

   stat_gradientinterval(mapping = aes(y=`DS_PANEL+Metabolomics`-`COX_PANELapoe4`), 
                          alpha=1, fatten_point=0.8, interval_size_range=c(0.3, 0.6), 
                          position = position_nudge(x = +0), slab_alpha=0)+

    annotate("text", x=1.2, y=-0.0105, position = position_nudge(x = +1),
             label="PANEL+MET", size=2)+


    stat_gradientinterval(mapping = aes(y=`DS_PANELapoe4+Metabolomics`-`COX_PANELapoe4`), 
                          alpha=1, fatten_point=0.8, interval_size_range=c(0.3, 0.6), 
                          position = position_nudge(x = -0.4), slab_alpha=0)+

    annotate("text", x=0.8, y=+0.0045, position = position_nudge(x = +1), label="PANEL+APOE4+MET", size=2)+



    scale_x_discrete(labels=em_wrap)+coord_flip(ylim=c(-0.03, 0.03))+
    scale_y_continuous()+ theme(plot.title = element_text(hjust=0.5))+theme(panel.grid.major.x=element_line(size=0.2), axis.text.y=element_blank(), axis.ticks.y=element_blank(), axis.line.y=element_blank())#labels = function(x) str_replace(x, '0(?=.)', ''))# + 

met_panel_apoe4

## Figure 4 - Additive Performance

## Trajectories

In [ ]:
run = "220126_apoe4"
partial_effects = arrow::read_feather(glue("{data_results_path}/adj_partial_effects_metabolomics_{run}.feather"))

In [ ]:
temp_prep = partial_effects %>% #filter(partition==0) %>% 
    filter(quantile %in% c(0.1, 0.5, 0.9)) %>% 
    filter(features %in% c("PANEL", "PANELapoe4")) %>%
    mutate(time=ntile(time, round(max(time)*10, 1))/10) %>% 
    group_by(endpoint, features, quantile, time) #%>% sample_n(27, replace=TRUE)

In [ ]:
purrr::map_dfr(integer(100), ~ temp_prep %>% sample_n(22, replace=TRUE), .id="i") -> temp_prep_bs
    
temp_prep_bs %>% head()

In [ ]:
unique(temp_prep_bs$features)

In [ ]:
ep_sel = c("M_all_cause_dementia")

temp_time = temp_prep %>% ungroup() %>% 
    filter(endpoint=="M_all_cause_dementia") %>% #sample_n(1000) %>%# mutate(time=ntile(time, round(max(time)*10, 1))/10) %>%
    group_by(endpoint, features, quantile, time) %>% 
    median_qi(Ft, na.rm=TRUE) # %>% ungroup() %>% sample(1000)
    
trajectories = ggplot(temp_time, aes(x = time, y=Ft, color=fct_rev(factor(quantile)), fill=fct_rev(factor(quantile)))) +
        #labs(title=endpoint_map[[endpoint]])+
        theme(plot.title = element_text(size=facet_size, hjust=0.5))+
        #geom_smooth(method="lm")+
        geom_lineribbon(aes(ymin = .lower, ymax = .upper), size=0.5)+
        #geom_kmticks(trans = "event", size=0.3) + geom_kmband(trans = "event") + 
        labs(x="Time [Years]", y="Adjusted Partial Trajectories [%]")+
        #geom_hline(aes(yintercept = max(Ft)), alpha=0.3, linetype="22")+
        #scale_color_manual(values=met_map)+scale_fill_manual(values=met_map)+
        scale_x_continuous(expand=c(0, 0))+
        scale_y_continuous(labels = function(x) round(x*100, 1), expand=c(0, 0))+
        scale_color_manual(values=c(`0.1`="#7AC6FF", `0.5`="#4F8EC1", `0.9`="#023768"))+ 
        scale_fill_manual(values=c(`0.1`=alpha("#7AC6FF", 0.3), `0.5`=alpha("#4F8EC1", 0.3), `0.9`=alpha("#023768", 0.3)))+                    
                           theme(legend.position="none")+               
        facet_grid(cols=vars(features), labeller=labeller(features=c("PANEL"="PANEL", "PANELapoe4"="PANEL+APOE4"))) #+ 
        #theme(strip.background = element_blank(), strip.text.x = element_blank())

In [ ]:
plot_width=4.25; plot_height=3; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
trajectories

In [ ]:
hrs = arrow::read_feather(glue("{data_results_path}/hrs_metabolomics_{run}.feather")) %>% 
    mutate(features = str_remove_all(str_sub(str_remove(str_remove(features, endpoint), "DS__"), end=-3), "_")) %>%
    filter(str_detect(covariate, "logh_"))

In [ ]:
library(zoo)

In [ ]:
library(ggdist)


In [ ]:
purrr::map_dfr(integer(1000), ~ hrs %>% group_by(endpoint, features) %>% sample_frac(1, replace=TRUE), .id="i") -> temp_hrs

In [ ]:
temp_hrs_agg = temp_hrs %>% filter(endpoint %in% ep_sel) %>%
    group_by(endpoint, features) %>% median_qi(`exp(coef)`, na.rm=TRUE) %>% rename(hr = `exp(coef)`) %>%
    mutate(hr = round(hr, 2), .lower=round(.lower, 2), .upper=round(.upper, 2)) %>%#c(hr, .lower, .upper), round, 1) %>%
    mutate(HR_Metabolomics = glue("{hr} ({.lower}, {.upper})")) %>% 
    select(endpoint, features, HR_Metabolomics) %>%
    pivot_wider(names_from="features", values_from="HR_Metabolomics") %>% 
    select(all_of(c("endpoint", 
                   "PANEL+Metabolomics", 'PANELapoe4+Metabolomics'))) %>% 
    mutate(endpoint = factor(endpoint, levels = endpoint_order)) %>% arrange(endpoint) %>%
    mutate(endpoint = recode(endpoint, !!!endpoint_map)) %>%
    ungroup()
temp_hrs_agg

In [ ]:
library(gt)

In [ ]:
fts_map = c(
    "Metabolomics" = "MET",
    "COX_Age+Sex" = "AgeSex",
    "Age+Sex+Metabolomics" = "AgeSex+MET",
    "COX_ASCVD" = "ASCVD",
    "ASCVD+Metabolomics" = "ASCVD+MET",
    "COX_PANEL" = "PANEL", 
    "PANEL+Metabolomics" = "PANEL+MET",
    "PANELapoe4+Metabolomics" = "PANEL+APOE4+MET"
               )

In [ ]:
hr_labels = temp_hrs_agg %>% 
    pivot_longer(-endpoint, names_to="features", values_to="HR")
hr_labels$fts_label = recode(hr_labels$features, !!!fts_map)
hr_labels

In [ ]:
temp = temp_hrs %>% filter(endpoint %in% ep_sel)   %>%  mutate(endpoint = factor(endpoint, levels = ep_sel)) %>% arrange(endpoint) %>%
    mutate(endpoint = recode(endpoint, !!!endpoint_map)) %>% filter(features %in% c("PANEL+Metabolomics", "PANELapoe4+Metabolomics")) %>%
    ungroup() 
temp %>% head()

plot_width=4; plot_height=1; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)

hr_plot = ggplot(temp, aes(x=fct_rev(factor(features, levels=c("PANEL+Metabolomics", "PANELapoe4+Metabolomics"))), y=`exp(coef)`))+#levels=endpoint_order_desc))) +
    labs(x=NULL, y="Adjusted Hazard Ratio / SD Metabolomic State")+
    #geom_boxplot()+
    stat_gradientinterval(color="black", alpha=0.7, fatten_point=0.8, interval_size_range=c(0.3, 0.6), fill="white")+

   # geom_quasirandom(mapping = aes(y=`DS_PANEL+Metabolomics`-`COX_PANEL`), color=alpha(color_map[['DS_Age+Sex+Metabolomics']], 0.1), size=0.1)+
    #geom_boxplot(mapping = aes(y=`DS_PANEL+Metabolomics`-`COX_PANEL`), 
    #             fill=alpha(color_map[['DS_PANEL+Metabolomics']], 0.1), width=0.5, alpha=0.7, size=0.3,
    #             outlier.shape=NA)+
    geom_hline(yintercept=1, alpha=0.7, color="black", size=0.25)+
    geom_hline(yintercept=1.5, alpha=0.3, color="black", size=0.25, linetype="22")+
    geom_hline(yintercept=2, alpha=0.4, color="black", size=0.25, linetype="22")+
    geom_hline(yintercept=3, alpha=0.5, color="black", size=0.25, linetype="22")+
    scale_x_discrete(labels=fts_map)+
    coord_flip(ylim=c(0.3, 2))+#ylim=c(-0.02, 0.02))+
    #scale_y_continuous()+ 
    theme(plot.title = element_text(hjust=0.5))+
    theme(strip.text.y.left = element_text(angle = 0))+
    theme(axis.text.y = element_text(size=5.5))+
    geom_text(data=hr_labels, mapping=aes(x=fct_rev(factor(features, levels=c("PANEL+Metabolomics", "PANELapoe4+Metabolomics"))), label=glue("HR[{fts_label}]: {HR}")), y=0.95, size=2, hjust=1, parse=TRUE) + 
    theme(axis.text.y = element_blank())
#panel.grid.major.y=element_line(size=0.2))#labels = function(x) str_replace(x, '0(?=.)', ''))# + 
    #annotate("text", x = 12, y = 0.18, label = "Age+Sex+Metabolomics", size=8, color=alpha(color_map[['DS_Age+Sex+Metabolomics']], 0.7))+
   # annotate("text", x = 12, y = 0.06, label = "Metabolomics", size=8, color=alpha(color_map[['DS_Metabolomics']], 0.7))+
    #annotate("text", x = 12, y = 0.113, label = "Age+Sex", size=8, color=alpha(color_map[['COX_Age+Sex']], 0.7))+
    #annotate("text", x = 11, y = 0.08, label = "Age+Sex", size=8, color=alpha(color_map[['COX_Age+Sex']], 0.7), hjust=0)+
    #annotate("text", x = 10, y = 0.08, label = "Metabolomics", size=8, color=alpha(color_map[['DS_Metabolomics']], 0.7), hjust=0)+
    #annotate("text", x = 8.6, y = 0.08, label = "Age+Sex+\nMetabolomics", size=8, color=alpha(color_map[['DS_Age+Sex+Metabolomics']], 0.7), hjust=0)
hr_plot

# Utility

In [ ]:
data_events = data %>% select(eid, ends_with("event"), ends_with("event_time")) %>% 
    pivot_longer(-eid, names_to=c("endpoint", "type"), values_to="value", names_pattern="(.*)(event_time|event)") %>% 
    mutate(endpoint = stringr::str_sub(endpoint, end=-2)) %>% pivot_wider(names_from="type", values_from="value")

In [ ]:
predictions_tte = predictions %>% left_join(data_events, by=c("endpoint", "eid"))

In [ ]:
library(ggthemes)
endpoint_map = c(
    'M_MACE'='MACE',
    'M_all_cause_dementia'='Dementia',
    'M_type_2_diabetes'='T2 Diabetes',
    'M_liver_disease'='Liver Disease',
    'M_renal_disease'='Renal Disease',
    'M_atrial_fibrillation'='Atrial Fibrillation',
    'M_heart_failure'= 'Heart Failure',
    'M_coronary_heart_disease'='CHD',
    'M_venous_thrombosis'='Ven. Thrombosis',
    'M_cerebral_stroke'='Cerebral Stroke',
    'M_abdominal_aortic_aneurysm'='AAA',
    'M_peripheral_arterial_disease'='PAD',
    "M_chronic_obstructuve_pulmonary_disease" = "COPD",
    "M_asthma" = "Asthma",
    'M_parkinsons_disease' = "Parkinson's",    
    "M_lung_cancer" = "Lung Cancer",
    "M_non_melanoma_skin_cancer" = "Skin Cancer",
    "M_colon_cancer"= "Colon Cancer",
    "M_rectal_cancer" = "Rectal Cancer",
    "M_prostate_cancer"= "Prostate Cancer",
    "M_breast_cancer" = "Breast Cancer",
    'M_cataracts' = "Cataracts", 
    'M_glaucoma' = "Glaucoma",
    'M_fractures' = "Fractures"
)

endpoint_order = c("M_MACE", "M_coronary_heart_disease", "M_cerebral_stroke", "M_all_cause_dementia", "M_heart_failure", "M_atrial_fibrillation",
                   "M_type_2_diabetes", "M_liver_disease", "M_renal_disease", "M_peripheral_arterial_disease", "M_venous_thrombosis",  "M_abdominal_aortic_aneurysm",
                   "M_chronic_obstructuve_pulmonary_disease", "M_asthma", 'M_parkinsons_disease', 'M_cataracts', 'M_glaucoma', 'M_fractures',
                    "M_lung_cancer","M_non_melanoma_skin_cancer","M_colon_cancer","M_rectal_cancer","M_prostate_cancer","M_breast_cancer"
                   
)

In [ ]:
library("jsonlite")
colors_path = "colors.json"
colors_dict = read_json(colors_path)

color_map <- c('COX_Age+Sex' = colors_dict$pastel$grey$light, 
               'DS_Metabolomics' = "#4F8EC1",  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = colors_dict$pastel$grey$mid, 
               'COX_PANEL' = colors_dict$pastel$grey$dark, 
               'COX_PANELapoe4' = colors_dict$pastel$grey$dark,
               'DS_Age+Sex+Metabolomics' = "#53dd6c",#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = "#d8315b",#colors_dict$pastel$red$mid,
               #'DS_QRISK3+Metabolomics' = colors_dict$pastel$red$mid,
               'DS_PANELnoblood+Metabolomics' = "forestgreen",
               'DS_PANEL+Metabolomics' = "#1e1b18", #colors_dict$pastel$red$dark
                'DS_PANELapoe4+Metabolomics' = "#1e1b18" #colors_dict$pastel$red$dark
      )

type_map <- c('COX_Age+Sex' = "dashed", 
               'DS_Metabolomics' = "solid",  #colors_dict$pastel$orange$dark,
               'COX_ASCVD' = "dashed", 
               'COX_PANEL' = "dotted", 
               'COX_PANELapoe4' = "dotted", 
               'DS_Age+Sex+Metabolomics' = "solid",#colors_dict$pastel$orange$mid,
               'DS_ASCVD+Metabolomics' = "solid",#colors_dict$pastel$red$mid,
               'DS_PANEL+Metabolomics' = "solid", #colors_dict$pastel$red$dark
                'DS_PANELnoblood+Metabolomics' = "solid",
               'DS_PANELapoe4+Metabolomics' = "solid",
              "Treat All" = "solid",
              "Treat None" = "solid"
      )

In [ ]:
scores_dca = c("DS_Metabolomics",
    "COX_PANEL",
    "DS_PANEL+Metabolomics",
     "COX_PANELapoe4",
    "DS_PANELapoe4+Metabolomics"

              )

In [ ]:
calculate_dca = function(endpoint){
    dca_temp = predictions_tte %>% filter(endpoint==!!endpoint) %>% 
        mutate(score=glue("{module}_{features}")) %>% 
        select(eid, score, Ft_10, event, event_time) %>% 
        filter(score %in% scores_dca) %>%
        pivot_wider(names_from="score", values_from="Ft_10") %>% 
        distinct()
    
    print(glue("{endpoint} n={nrow(dca_temp)}"))
    flush.console()
    
    dca_endpoint = dca(
        Surv(event_time, event) ~ DS_Metabolomics + COX_PANEL + `DS_PANEL+Metabolomics` + `COX_PANELapoe4` + `DS_PANELapoe4+Metabolomics`, 
        data = dca_temp,
        time = 10,
        thresholds = seq(0, 0.5, by=0.001)
           )$dca
    return(dca_endpoint %>% mutate(st_net_benefit = net_benefit/prevalence))
}

In [ ]:
library(dcurves)
dca_data = calculate_dca("M_all_cause_dementia")

## AgeSex + Metabolomics vs PANEL

In [ ]:
scores_plot = c(
    "Treat All", 
    "Treat None",
    "COX_PANEL",
    "COX_PANELapoe4",
    "DS_PANEL+Metabolomics",
    "DS_PANELapoe4+Metabolomics"
)

In [ ]:
scores_plot = c(
    "Treat All", 
    "Treat None",
    "COX_PANEL",
   "DS_PANEL+Metabolomics",
    "COX_PANELapoe4", 
    "DS_PANELapoe4+Metabolomics"
)

endpoint="M_all_cause_dementia"
temp = dca_data

endpoint_label = endpoint_map[[endpoint]]
    temp_plot = temp %>% filter(label %in% scores_plot)
temp_plot_wide = temp_plot %>%  select(threshold, label, st_net_benefit) %>% mutate(st_net_benefit = st_net_benefit*100) %>% pivot_wider(names_from="label", values_from="st_net_benefit")

plotA = 
    ggplot(temp_plot)+
      labs(x="Threshold Probability [%]", y="Standardized Net Benefit [%]")+

    #geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=0, ymax=`COX_PANEL`), fill="black", alpha=0.3, colour = NA)+       
    geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`COX_PANEL`, ymax=`DS_PANEL+Metabolomics`), fill="firebrick", alpha=0.7, colour = NA)+
    #geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`DS_PANEL+Metabolomics`, ymax=`COX_PANELapoe4`), fill="black", alpha=0.05, colour = NA)+
    geom_ribbon(data=temp_plot_wide, mapping=aes(x=threshold, ymin=`COX_PANELapoe4`, ymax=`DS_PANELapoe4+Metabolomics`), fill="firebrick", alpha=0.7, colour = NA)+
geom_line(data=temp_plot, mapping=aes(x=threshold, y=st_net_benefit*100, color=label, linetype=label), size=0.25) +
      coord_cartesian(xlim=c(0, 0.1), ylim=c(0, 100))+
      scale_x_continuous(labels=function(x) x*100, expand=c(0, 0))+
      scale_color_manual(values=color_map)+
      scale_linetype_manual(values=type_map)+
    theme(legend.position="none", panel.grid.major.y = element_line())

In [ ]:
plot_width=8.25; plot_height=6; plot_dpi=320
options(repr.plot.width = plot_width, repr.plot.height = plot_height, repr.plot.res=plot_dpi)
cox = (trajectories/hr_plot + plot_layout(height=c(6, 1))) 
fig_apoe4 = (met_panel_apoe4 + annotate("text", x=1.75, y=0.00005, position = position_nudge(x = +1), label="PANEL + APOE4", size=2))/(cox|plotA) + plot_layout(height=c(1, 6))
fig_apoe4

In [ ]:
plot_name = "Suppl_Figures_6_APOE4"
fig_apoe4 %>% ggsave(filename=glue("outputs/{plot_name}.pdf"), device=cairo_pdf, width=plot_width, height=plot_height, dpi=plot_dpi)